In [ ]:
import pandas as pd
from konlpy.tag import Mecab
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
def tokenize_sentense(text): # 형태소 변환 함수
    mecab=Mecab()
    return mecab.morphs(text)

In [1]:
raw_data = pd.read_csv('spam.csv', index_col = 0) # 데이터 기본 전처리
raw_data.reset_index(drop = True, inplace = True)
raw_data.dropna(axis = 0)

NameError: ignored

# 기본 전처리

In [ ]:
raw_data['token'] = raw_data['text'].apply(tokenize_sentense) # text 형태소 변환
raw_data.head()

In [ ]:
raw_data['label'] = raw_data['label'].map({'spam' : 1, 'ham' : 0}) # label열 가변수화
raw_data.head()

In [ ]:
# 기본 전처리가 끝난 데이터를 저장
# raw_data.to_csv('spam_preprocessing.csv')

In [ ]:
raw_data.drop_duplicates(subset=['text'], inplace=True) # text열 중복 제거
print('총 샘플의 수 :',len(raw_data))
train_data, test_data = train_test_split(raw_data, test_size = 0.3, random_state = 2022) # random state 값은 상관 X
print('훈련용 데이터의 개수 :', len(train_data))
print('테스트용 데이터의 개수 :', len(test_data))

In [ ]:
train_data['label'].value_counts().plot(kind = 'bar') # 1 (spam), 0 (ham)의 개수 파악

# 정수 임베딩

In [ ]:
X_train = train_data['token'].values # dict에서 값을 뽑음, 아마 토큰화된 리스트 속 리스트의 단어 하나하나를 뽑는 것 같음
y_train = train_data['label'].values
X_test= test_data['token'].values
y_test = test_data['label'].values

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train) # 정수 임베딩

In [ ]:
threshold = 2 # 희귀단어 제거 (등장 횟수가 2회 미만인 단어 제거)
total_cnt = len(tokenizer.word_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0


for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value


    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2 (OOV = out of vocabulary, 컴퓨터가 이해할 수 없는 단어)
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') # 단어 집합의 크기보다 큰 숫자가 부여된 단어들은 전부 OOV로 변환
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
print(X_train[:3]) # 정수 임베딩 적용 확인

# 패딩

In [ ]:
print('메세지의 최대 길이 :',max(len(message) for message in X_train)) # 전체 데이터의 분포 확인
print('메세지의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(message) for message in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
def below_threshold_len(max_len, nested_list): # 패딩을 max_len으로 했을때 얼마나 많은 샘플을 보전할 수 있는지 확인
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

max_len = 70
below_threshold_len(max_len, X_train)

In [ ]:
X_train = pad_sequences(X_train, maxlen=max_len) # max_len으로 train, test padding
X_test = pad_sequences(X_test, maxlen=max_len)

# Bahdanau attention
- 바다나우 어텐션은 RNN의 근본적인 문제, time step을 지나며 데이터가 손실되는 것을 해결하기 위해 RNN의 모든 은닉 상태를 다시 한 번 참고해 주는 방법임
- 어텐션 메커니즘에는 기본적으로 루옹 어텐션이 있고 바다나우가 조금 더 복잡한 형태임 (바다나우 어텐션 이론 참고 : https://wikidocs.net/73161)
- 루옹 어텐션, 닷 프로덕트(루옹 어텐션 중 하나)와의 차이점 (https://velog.io/@guide333/Attention-%EC%A0%95%EB%A6%AC#%EB%B0%94%EB%8B%A4%EB%82%98%EC%9A%B0-%EC%96%B4%ED%85%90%EC%85%98%EA%B3%BC-%EB%A3%A8%EC%98%B9-%EC%96%B4%ED%85%90%EC%85%98%EC%9D%98-%EC%B0%A8%EC%9D%B4)

In [ ]:
import tensorflow as tf

class BahdanauAttention(tf.keras.Model): # Bahdanau attention 함수
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

# 모델링

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

sequence_input = Input(shape=(max_len,), dtype='int32') # 10000개의 단어를 128차원의 벡터로 임베딩하도록 설계
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero = True)(sequence_input)

# lstm 두 층을 쌓음

lstm = Bidirectional(LSTM(128, dropout=0.5, return_sequences = True))(embedded_sequences)
# lstm 첫번째 층, 두 번째 층을 위에 쌓을 예정이므로 return_sequences를 True로 해주어야 함 (아래층이 forward 방향이고 위층이 backward 방향이라는 것을 뜻하는 것 같음)
# 양방향 lstm 구조 참고 (https://wegonnamakeit.tistory.com/25) - 그림 8-30
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(128, dropout=0.5, return_sequences=True, return_state=True))(lstm)
# lstm 두 번째 층
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)
# 각 상태의 shape 출력

In [ ]:
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)
# 컨텍스트 벡터는 은닉 상태와 어텐션 가중치를 가중합하여 구할 수 있음 (https://velog.io/@eunddodi/바다나우-어텐션) - 3)
dense1 = Dense(20, activation="relu")(context_vector) # 컨텍스트 벡터를 dense층에 통과
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout) # 분류 문제이므로 sigmoid, 출력은 1
model = Model(inputs=sequence_input, outputs=output)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model_bahdanau.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
# 모델 중간저장
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs = 15, callbacks=[es, mc], batch_size = 256, validation_split=0.2, verbose=1)

In [ ]:
test_data = pd.read_csv('spam_test_text.csv')

test_data['token'] = test_data['text'].apply(tokenize_sentense) # test 데이터 토큰화

test_data.head()

In [ ]:
test = tokenizer.texts_to_sequences(test_data['token'].values) # test 데이터 정수 임베딩
test = pad_sequences(test, maxlen=max_len) # test 데이터 패딩
test[:3]

In [ ]:
score = model.predict(test) # 예측

In [ ]:
score = pd.DataFrame(score)
score.columns = ['result1']

In [ ]:
score.loc[score['result1'] >= 0.5, 'result2'] = 'spam' # 0.5 이상은 1
score.loc[score['result1'] < 0.5, 'result2'] = 'ham' # 0.5 미만은 0
df = pd.DataFrame()
df['id'] = list(range(9896))
df['label'] = score['result2']
df

In [ ]:
df.to_csv('result_4.csv', index = False)